In [48]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import urllib3

# Disable insecure request warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Make an HTTP request to the webpage
url = 'https://clashroyale.fandom.com/wiki/Cards'
response = requests.get(url, verify=False)

# Check the response code
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = bs(response.text, 'html.parser')

    # Identify the parent div element that contains all the div tables
    parent_div = soup.find('div', attrs={'class': 'main-container'})

    parent_div = soup.find('div', attrs={'class':'main-container'})

    # Check if the parent div element exists
    if parent_div:
        # Find all the div tables within the parent div element
        div_tables = parent_div.find_all('div', attrs={'class': 'table-back'})

        # Initialize lists to hold the data and column headers
        data = []
        headers = []

        # Extract the table headers (column names)
        headers_row = div_tables[0].find('tr')
        if headers_row:
            headers = headers_row.find_all(['th'])
            headers = [header.get_text().strip() for header in headers]

        # Iterate through each div table
        for div_table in div_tables:
            # Extract the table data (rows and columns) from each div table
            rows = div_table.find_all('tr')
            for row in rows:
                columns = row.find_all(['th', 'td'])
                row_data = [column.get_text().strip() for column in columns]

                # Pad the row data with empty strings if the number of columns is less than headers
                while len(row_data) < len(headers):
                    row_data.append('')

                # Truncate the row data if it has more columns than headers
                row_data = row_data[:len(headers)]

                data.append(row_data)

        # Check the extracted data and headers
        print("Headers:", headers)
        for row in data:
            print("Row:", row)

        # Create a DataFrame using pandas
        df = pd.DataFrame(data, columns=headers)

        # Save the DataFrame to an Excel file
        df.to_excel('troop_data.xlsx', index=False)
        print("Data exported to troop_data.xlsx successfully.")

    else:
        print("Parent div element not found.")
else:
    print(f"Failed to fetch the webpage. Status Code: {response.status_code}")

Headers: ['Card', 'Cost', 'Health (+Shield)', 'Damage', 'Hit Speed (seconds)', 'Damage per Second', 'Spawn/Death Damage', 'Range', 'Count', 'Special']
Row: ['Card', 'Cost', 'Health (+Shield)', 'Damage', 'Hit Speed (seconds)', 'Damage per Second', 'Spawn/Death Damage', 'Range', 'Count', 'Special']
Row: ['Archers', '3', '304', '107', '0.9', '118', '0', '5', '2', 'N/A']
Row: ['Archer Queen', '5', '1,000', '225', '1.2', '188', '0', '5', '1', 'Champion Ability']
Row: ['Baby Dragon', '4', '1,152', '160', '1.5', '107', '0', '3.5', '1', 'N/A']
Row: ['Balloon', '5', '1,680', '640', '2', '320', '240 (Death)', '0.1 (Melee: Short)', '1', 'N/A']
Row: ['Bandit', '3', '907', '193', '1', '193', '0', '0.75 (Melee: Short)', '1', 'N/A']
Row: ['Barbarians', '5', '670', '192', '1.4', '137', '0', '0.7 (Melee: Short)', '5', 'N/A']
Row: ['Bats', '2', '81', '81', '1.3', '62', '0', '1.2 (Melee: Medium)', '5', 'N/A']
Row: ['Battle Healer', '4', '1,717', '148', '1.5', '98', '0', '1.6 (Melee: Long)', '1', '66 (16.